In [ ]:
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet #无效
import nltk
from nltk.corpus import words
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import swifter #并行处理pandas
import re

In [ ]:
text1='The GPS system is one of the widely used methods to determine the real-time position in navigation applications. Using this system requires error compensation which effectively improves the positioning accuracy. One of the errors that needs to be compensated is the tropospheric signal transmission delay from the satellite to the GPS receiver. Tropospheric delay appears as bias in measuring pseudo-range by the GPS receiver and results in a constant deviation in determining final position of the user receiver. Several tropospheric models have been developed and employed to compensate and correct tropospheric delay. In this paper, some of the conventional tropospheric delay models and a combination of several models have been investigated and employed to determine the real-time position. These models are applied to the real-time positioning algorithm of the single frequency GPS receiver and have been compared to each other. This comparison is based on a reduction in the error in estimating the final position of the receiver, and a decrease in the computation time of the real-time positioning. Real-time positioning tests are performed in 6 iterations throughout six days. Positioning results are compared with the triple frequency GPS receiver GINTEC -G10. Comparison results show that real-time positioning of single frequency GPS receiver has sufficient and better accuracy using "Modified Saastamoinen" model and "Modified Hopfield" model have enough accuracy for low cost real-time positioning in most iterations of the experiment, and "Modified Hopfield" model is more suitable for low-cost real-time positioning, because its computation time is shorter.'
text2='Ensuring the accurate positioning of the vehicle at any time is the key to navigation and path planning in unmanned driving vehicle. Due to the changes and signal attenuation of vehicle-mounted GPS signals when they pass through electric clouds, and the reflection of GPS signals when they encounter glass curtain wall, and under underground garages, tunnels and overpasses, for example, GPS satellite signals cannot be received by vehicle-mounted GPS locator due to the obstruction of buildings. The above-mentioned situations will cause the vehicle GPS positioning signal to be unstable or even missing. And the current positioning method based on lidar has become the mainstream positioning method for unmanned vehicles. However, under bad weather conditions, the detection capability of infrared waves will be greatly attenuated, and the stability of lidar positioning will also be affected. Therefore, this paper introduces an unmanned driving vehicle positioning method based on GPS/lidar/derivative data fusion when the environment map is known. That is, when the GPS signal is good and the lidar is in good weather conditions, the vehicle can be accurately positioned by GPS/lidar/derivation fusion. While the GPS positioning signal or lidar positioning is unstable and in the event of a short absence, it is still can achieve accurate positioning of intelligent vehicles through a derivation-based approach. The experimental data and MATLAB software simulation results show that the horizontal error of vehicle positioning is within the range of 14.3cm and the longitudinal error is within the range of 8.8cm, within 5 seconds after the lidar and GPS signals are lost simultaneously.'
text3='In this paper, we present an indoor positioning system based on GPS repeaters. Each repeater consists of a directional antenna for receiving a non-overlapping set of GPS satellites, a LNA and a power amplifier for compensating the antenna and cable losses, and a transmitting antenna for re-radiating the amplified GPS signals. The proposed system has been shown to be very successful for 1D positioning using a set of two repeaters. Similarly, successful results have been obtained for 2D positioning using three repeaters.'
text4='The effect of radio frequency interference (RFI) on Global Positioning System (GPS) signals needs to be investigated because of the increasing demand for accuracy and precision in positioning, navigation, and timing (PNT) applications. In order to investigate the effect of RFI on the high precision GPS applications (e.g., surveying works), this study was conducted on single- and dual-frequency GPS receivers in differential GPS (DGPS) mode via static observations. All the tests were focused on in-band interference at the GPS L1 and L2 frequency bands with the interference signal power levels being increased from -140 dB until the accuracy of GPS was affected. The use of dilution of precision (DOP) alone to quantify the quality of GPS was found to be insufficient, and hence, residual positions have been used to study this effect. The findings from all the tests show that RFI can degrade the accuracy of GPS measurement, and, in some cases, completely disrupt GPS signals (e.g., loss of tracking of GPS signals). Hence, the presence of RFI sources, both intentional and unintentional, should be an important consideration in planning high precision GPS application.'
abstract_list=[text1,text2,text3,text4]

In [ ]:
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag): # 词性转化成 WordNetLemmatizer 可识别的
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def rm_nnp(text=''):
    taged_word_list=nltk.pos_tag(text.lower().split())
    return [lemmatizer.lemmatize(token, get_wordnet_pos(tag)) for token, tag in taged_word_list if tag != 'NNP']

abstract_word_list=[rm_nnp(text) for text in abstract_list]
print(abstract_word_list)

In [ ]:
word_list=sum(abstract_word_list,[])
word_counts = Counter(word_list)
print(word_counts)

In [ ]:
word_pd= pd.DataFrame(word_counts.items(), columns=['word', 'count'])
word_pd = word_pd.sort_values('count', ascending=False).reset_index(drop=True)
word_pd.head(5)

In [ ]:
word_pd.info()

In [ ]:
q1 = word_pd['count'].quantile(0.2)
q2 = word_pd['count'].quantile(0.995)
word_pd_80 = word_pd[(word_pd['count'] >= q1) & (word_pd['count'] <= q2)].reset_index(drop=True)
word_pd_80.head(3)

In [ ]:
word_pd_80.info()

In [ ]:
article_num = len(article_list) #计算文档总数

#df_temp=dataset['content_list'].apply(lambda x: word_pd_60['word'].isin(x).astype(int)).sum() 

df_temp=np.sum([word_pd_60['word'].isin(text_list).astype(int) for text_list in abstract_word_list])
df_temp=np.array(df_temp)

idf=np.log(article_num/df_temp) #计算idf
Idf_Dict=dict(zip(word_pd_60['word'],idf))
print(Idf_Dict)
Idf=pd.DataFrame(list(Idf_Dict.items()), columns=['word', 'idf'])
Idf.head(3)